## Sage Data Merge

dupes:
    - yQ7pYy
    - ksJM3Y
    

In [1]:
import os, pandas as pd, numpy as np, re
from functools import reduce
pd.set_option('display.max_columns', 100)

#NOTE TO SELF: redo the relative paths.
sage = '/home/wraikes/Dropbox/partnership/DMTdata_8_31_17/sage'
os.chdir(sage)

In [2]:
#GLOBAL VARIABLE
new_dfs = {}

#Functions for data processing
def remove_dupes(df):
    
    test_users = [
        'ThpMV2Achc', 
        'SEkQVTCe6j', 
        'Wh8NSX3DHL', 
        'SaXFr2kPZa', 
        'VWUcSp4TeH', 
        'yXEfAmW682', 
        'gwEpQR8j9B',
        'WbbNWM4RAF', 
        'D5bzYrfd8E', 
        'LJcmEFWp74', 
        'ULoF3MM1nN',
        'yQ7pYy'
    ]

    diff_study = [
        'rL8eA3',
        'rLg5xs',
        'rLrD9h',
        'rLP7H2',
        'rL6s6h',
        'aOyzBg',
        'aORA43',
        'aOh48U',
        'aOLu4K',
        'aOQtxv',
        'aO5TvQ',
        'mPC9S8',
        'mPgquX',
        'mP5xkB',
        'mPSQvh',
        'mPYk2p',
        'mP3rbd'
    ]

    
    return df[~df.externalId.isin(test_users + diff_study)]

def dupe_check(df):
    return len(df.externalId) == len(df.externalId.unique()) and len(df.externalId) > 0

def new_cols(df, append):
    # Remove: 'metadata.json.' and 'data.json.'.
    col_re = re.compile('metadata.json.|data.json.')
    df.columns = list(map(lambda x: re.sub(col_re, '', x), df.columns))
    
    # Append df identifier to columns.
    df.columns = list(map(lambda x: append + x, df.columns))
    df = df.rename(columns = {append + 'externalId': 'externalId'})

    return df

def create_new_df(df, att, var_1, var_2=None):
    cols = df.columns
    
    new_df = pd.DataFrame(columns = cols)
        
    for ix, row in df.iterrows():
        if var_2:
            if row[var_1] == att[1] and row[var_2] == att[2]:
                new_df = new_df.append(row, ignore_index=True)
        else:
            if row[var_1] == att[1]:
                new_df = new_df.append(row, ignore_index=True)
    
    return new_df

def df_merge(df, attributes, var_1, var_2=None):
    dfs = []
    
    for att in attributes:
        new_df = create_new_df(df, att, var_1, var_2)
        new_df = new_cols(new_df, att[0])
        dfs.append(new_df)
    
    df_merge = reduce(lambda left, right: pd.merge(left, 
                                                   right, 
                                                   how = 'outer',
                                                   on='externalId'),
                      dfs)
    
    return df_merge

def final_check(df, name):
    if dupe_check(df):
        new_dfs[name] = df
        print('Done!')
    else:
        print('False')

### Process files that do not need cleaning / restructuring.

In [3]:
files_to_exclude = ['digital-marshmallow-status_8.31.17.csv',
                    'digital-marshmallow-appVersion_8.31.17.csv']

files_as_is = []
files_to_modify = []

for file in os.listdir():
    if file not in files_to_exclude:
        df = pd.read_csv(file)
        df = remove_dupes(df)
        if dupe_check(df):
            files_as_is.append(file)
        else:
            files_to_modify.append(file)

files_as_is.sort()

In [4]:
len(files_as_is)

10

In [5]:
files_to_modify

['digital-marshmallow-morning_notification_time-v3_8.31.17.csv',
 'digital-marshmallow-bart-v4_8.31.17.csv',
 'digital-marshmallow-goNoGo-v2_8.31.17.csv',
 'digital-marshmallow-morning_yesterday_sem_diff-v2_8.31.17.csv',
 'digital-marshmallow-delay_discounting_raw-v6_8.31.17.csv',
 'digital-marshmallow-evening_sem_diff-v2_8.31.17.csv',
 'digital-marshmallow-comments-v2_8.31.17.csv',
 'digital-marshmallow-discounting_raw-v2_8.31.17.csv',
 'digital-marshmallow-morning_sem_diff-v2_8.31.17.csv',
 'digital-marshmallow-pam-v2_8.31.17.csv',
 'digital-marshmallow-morning_behavior_choices-v1_8.31.17.csv',
 'digital-marshmallow-pam_multiple-v2_8.31.17.csv',
 'digital-marshmallow-behavior_choices_4_bl-v2_8.31.17.csv',
 'digital-marshmallow-evening_notification_time-v2_8.31.17.csv',
 'digital-marshmallow-morning_yesterday_likert-v2_8.31.17.csv']

In [6]:
files_as_is

['digital-marshmallow-as_a_child_likert_bl-v2_8.31.17.csv',
 'digital-marshmallow-behavior_choices_1_bl-v1_8.31.17.csv',
 'digital-marshmallow-behavior_likert_21-v2_8.31.17.csv',
 'digital-marshmallow-behavior_likert_bl-v2_8.31.17.csv',
 'digital-marshmallow-comments_21-v2_8.31.17.csv',
 'digital-marshmallow-demographics-v2_8.31.17.csv',
 'digital-marshmallow-generally_sem_diff_21-v2_8.31.17.csv',
 'digital-marshmallow-generally_sem_diff_bl-v2_8.31.17.csv',
 'digital-marshmallow-past_year_likert_21-v2_8.31.17.csv',
 'digital-marshmallow-past_year_likert_bl-v3_8.31.17.csv']

In [7]:
# CHECK TO MAKE SURE FILE NAMES SYNC UP WITH files_as_is
names_as_is = [
    'SAGE_as_a_child_-_',
    'SAGE_behavior_choices_1_-_',
    'SAGE_behavior_lk_21_-_',
    'SAGE_behavior_lk_bl_-_',
    'SAGE_comments_21_-_',
    'SAGE_demos_-_',
    'SAGE_generally_sem_21_-_',
    'SAGE_generally_sem_bl_-_',
    'SAGE_past_year_likert_21_-_',
    'SAGe_past_year_likert_bl_-_'
]

# names_as_is = [
#     'SAGE_generally_sem_bl_-_',
#     'SAGe_past_year_likert_bl_-_',
#     'SAGE_as_a_child_-_',
#     'SAGE_behavior_lk_21_-_',
#     'SAGE_past_year_likert_21_-_',
#     'SAGE_comments_21_-_',
#     'SAGE_generally_sem_21_-_',
#     'SAGE_behavior_lk_bl_-_',
#     'SAGE_behavior_choices_1_-_',
#     'SAGE_demos_-_'
# ]

In [8]:
for file, name in zip(files_as_is, names_as_is):
    df = pd.read_csv(file)
    df = remove_dupes(df)
    df = new_cols(df, name)
    new_dfs[name] = df

In [9]:
all([dupe_check(df) for name, df in new_dfs.items()])

True

### Dupe: yQ7pYy

In [ ]:
demos = pd.read_csv('digital-marshmallow-demographics-v2_8.31.17.csv')
demos = remove_dupes(demos)

dupe_check(demos)

In [ ]:
demos.externalId[demos.externalId.duplicated()]

In [ ]:
gen = pd.read_csv('digital-marshmallow-generally_sem_diff_bl-v2_8.31.17.csv')
gen = remove_dupes(gen)

dupe_check(gen)

In [ ]:
gen.externalId[gen.externalId.duplicated()]

In [ ]:
gen[gen.externalId == 'yQ7pYy']

In [ ]:
demos[demos.externalId == 'yQ7pYy']

### DataFrame: Bart_V4

In [10]:
bart_v4 = pd.read_csv('digital-marshmallow-bart-v4_8.31.17.csv')
bart_v4 = remove_dupes(bart_v4)

dupe_check(bart_v4)

False

In [11]:
bart_attributes = [
    ['SAGE_bart_bl_0.25_-_', 'baseline', 'BART0.25'],
    ['SAGE_bart_bl_250_-_', 'baseline', 'BART250.00'],
    ['SAGE_bart_21_0.25_-_', '21-day-assessment', 'BART0.25'],
    ['SAGE_bart_21_250_-_', '21-day-assessment', 'BART250.00']
]

In [12]:
bart_v4 = df_merge(bart_v4, 
                   bart_attributes,
                   var_1='metadata.json.taskIdentifier',
                   var_2='data.json.variable_label')

In [13]:
final_check(bart_v4, 'bart_v4')

Done!


### DataFrame: Behavior_choices_4

In [14]:
behavior_4 = pd.read_csv('digital-marshmallow-behavior_choices_4_bl-v2_8.31.17.csv')
behavior_4 = remove_dupes(behavior_4)

dupe_check(behavior_4)

False

In [15]:
behavior_4_attributes = [
    ['SAGE_behavior_4_bl_-_', 'baseline'],
]

In [16]:
behavior_4 = df_merge(behavior_4, 
                      behavior_4_attributes,
                      var_1='metadata.json.taskIdentifier')

In [17]:
final_check(behavior_4, 'behave_4')

Done!


### DataFrame: Delay Discounting

In [18]:
delay = pd.read_csv('digital-marshmallow-delay_discounting_raw-v6_8.31.17.csv')
delay = remove_dupes(delay)

dupe_check(delay)

False

In [19]:
bl = 'baseline'
_21 = '21-day-assessment'

delay_attributes = [
    ['SAGE_delay_bl_time_6_month_-_', bl, 'dd_time_6_month'],
    ['SAGE_delay_bl_money_6_month_-_', bl, 'dd_money_6_month'],
    ['SAGE_delay_bl_money_1_month_-_', bl, 'dd_money_1_month'],
    ['SAGE_delay_bl_time_1_year_-_', bl, 'dd_time_1_year'],
    ['SAGE_delay_21_time_6_month_-_', _21, 'dd_time_6_month'],
    ['SAGE_delay_21_money_6_month_-_', _21, 'dd_money_6_month'],
    ['SAGE_delay_21_money_1_month_-_', _21, 'dd_money_1_month'],
    ['SAGE_delay_21_time_1_year_-_', _21, 'dd_time_1_year']
]

In [20]:
delay = df_merge(delay, 
                 delay_attributes, 
                 var_1='metadata.json.taskIdentifier',
                 var_2='data.json.variableLabel')

In [21]:
final_check(delay, 'delay')

Done!


### DataFrame: Discounting Raw

In [22]:
discount = pd.read_csv('digital-marshmallow-discounting_raw-v2_8.31.17.csv')
discount = remove_dupes(discount)

dupe_check(discount)

False

In [23]:
bl = 'baseline'
_21 = '21-day-assessment'

discount_attributes = [
    ['SAGE_discount_bl_money_-_', bl, 'pd_constant_money'],
    ['SAGE_discount_bl_prob_-_', bl, 'pd_constant_probabiliy'],
    ['SAGE_discount_21_money_-_', _21, 'pd_constant_money'],
    ['SAGE_discount_21_prob_-_', _21, 'pd_constant_probability']
]

discount = df_merge(discount, 
                    discount_attributes,
                    var_1='metadata.json.taskIdentifier',
                    var_2='data.json.variableLabel')

In [24]:
final_check(discount, 'discount')

Done!


### DataFrame: Evening Notification

In [25]:
evening_note = pd.read_csv('digital-marshmallow-evening_notification_time-v2_8.31.17.csv')
evening_note = remove_dupes(evening_note)

dupe_check(evening_note)

False

In [26]:
evening_note_attributes = [
    ['SAGE_evening_note_bl_-_', 'baseline']
]

evening_note_bl = df_merge(evening_note, 
                           evening_note_attributes,
                           var_1='metadata.json.taskIdentifier')

In [27]:
final_check(evening_note_bl, 'evening_note')

Done!


### DataFrame: GoNoGo

In [28]:
gonogo = pd.read_csv('digital-marshmallow-goNoGo-v2_8.31.17.csv')
gonogo = remove_dupes(gonogo)

dupe_check(gonogo)

False

#### Remove ksJM3Y until further notice.

In [29]:
gonogo[gonogo.externalId == 'ksJM3Y']

,ROW_ID,ROW_VERSION,recordId,appVersion,phoneInfo,uploadDate,healthCode,externalId,dataGroups,createdOn,createdOnTimeZone,userSharingScope,metadata.json.UUID,metadata.json.taskIdentifier,metadata.json.taskRunUUID,metadata.json.startDate,metadata.json.startDate.timezone,metadata.json.endDate,metadata.json.endDate.timezone,metadata.json.groupLabel,data.json.variable_label,data.json.number_of_trials,data.json.number_of_correct_responses,data.json.number_of_correct_responses_first_third,data.json.number_of_correct_responses_second_third,data.json.number_of_correct_responses_last_third,data.json.number_of_correct_nonresponses,data.json.number_of_correct_nonresponses_first_third,data.json.number_of_correct_nonresponses_second_third,data.json.number_of_correct_nonresponses_last_third,data.json.number_of_incorrect_responses,data.json.number_of_incorrect_responses_first_third,data.json.number_of_incorrect_responses_second_third,data.json.number_of_incorrect_responses_last_third,data.json.number_of_incorrect_nonresponses,data.json.number_of_incorrect_nonresponses_first_third,data.json.number_of_incorrect_nonresponses_second_third,data.json.number_of_incorrect_nonresponses_last_third,data.json.response_time_mean,data.json.response_time_mean_first_third,data.json.response_time_mean_second_third,data.json.response_time_mean_last_third,data.json.response_time_range,data.json.response_time_range_first_third,data.json.response_time_range_second_third,data.json.response_time_range_last_third,data.json.response_time_std_dev,data.json.response_time_std_dev_first_third,data.json.response_time_std_dev_second_third,data.json.response_time_std_dev_last_third,data.json.response_time_mean_after_one_incorrect,data.json.response_time_mean_after_one_incorrect_first_third,data.json.response_time_mean_after_one_incorrect_second_third,data.json.response_time_mean_after_one_incorrect_last_third,data.json.response_time_mean_after_ten_correct,data.json.response_time_mean_after_ten_correct_first_third,data.json.response_time_mean_after_ten_correct_second_third,data.json.response_time_mean_after_ten_correct_last_third,data.json.response_time_mean_correct,data.json.response_time_mean_correct_first_third,data.json.response_time_mean_correct_second_third,data.json.response_time_mean_correct_last_third,data.json.response_time_mean_incorrect,data.json.response_time_mean_incorrect_first_third,data.json.response_time_mean_incorrect_second_third,data.json.response_time_mean_incorrect_last_third,data.json.researcher_code
39,39,6,915668b8-d864-4ac3-95c5-e2f2f32475db,"version 1.0, build 16",iPhone 6; iOS/10.2.1,6/9/2017,65bfff61-e5fe-417c-8b8a-da43c48328a8,ksJM3Y,NaN,1.496950e+12,-400,ALL_QUALIFIED_RESEARCHERS,C93834E7-341C-4414-81D2-B43B740E1C33,baseline,A5645017-C58C-495A-A64D-56C05AA9256B,1.496950e+12,-400,1.496950e+12,-400,14,go_no_go_stable_stimulus_active_task,75,33,10,10,13,40,14,14,12,1,1,0,0,1,0,1,0,333.021883,330.909724,337.812805,331.123769,162.079036,162.079036,88.642061,138.779998,37.371420,41.322636,27.778881,39.865079,493.188977,236.377954,750.000000,0.000000,561.724514,750.000000,373.449027,0.000000,335.950487,340.362900,337.812805,331.123769,236.377954,236.377954,0.000000,0.000000,NaN
43,43,6,f2b53179-389d-4ada-8cf6-2e15dd73a42d,"version 1.0, build 16",iPhone 6; iOS/10.2.1,6/9/2017,65bfff61-e5fe-417c-8b8a-da43c48328a8,ksJM3Y,NaN,1.496950e+12,-400,ALL_QUALIFIED_RESEARCHERS,16C4B50E-4950-407A-AEDB-C4618BB0F360,baseline,A5645017-C58C-495A-A64D-56C05AA9256B,1.496950e+12,-400,1.496950e+12,-400,14,go_no_go_variable_stimulus_active_task,75,43,15,15,13,30,10,9,11,2,0,1,1,0,0,0,0,324.080139,343.706731,338.511758,286.558368,533.311963,267.641962,137.221038,369.893968,70.652764,69.216745,32.169978,87.581776,143.344998,0.000000,272.265971,14.424026,328.355521,343.138039,313.573003,0.000000,332.486424,343.706731,342.928143,307.491779,143.344998,0.000000,272.265971,14.424026,NaN
60,60,8,4641f7be-713d-475a-be00-a769f12bc855,"version 1.0, build 16",iPhone 6; iOS/10.2.1,6/

In [30]:
gonogo.drop([669], inplace=True)

##### ksJM3Y

In [31]:
bl = 'baseline'
_21 = '21-day-assessment'

gonogo_attributes = [
    ['SAGE_gonogo_bl_stable_-_', bl, 'go_no_go_stable_stimulus_active_task'],
    ['SAGE_gonogo_21_variable_-_', bl, 'go_no_go_variable_stimulus_active_task'],
    ['SAGE_gonogo_bl_stable_-_', _21, 'go_no_go_stable_stimulus_active_task'],
    ['SAGE_gonogo_21_variable_-_', _21, 'go_no_go_variable_stimulus_active_task']
]

gonogo = df_merge(gonogo, 
                  gonogo_attributes,
                  var_1='metadata.json.taskIdentifier',
                  var_2='data.json.variable_label')

In [32]:
final_check(gonogo, 'gonogo')

Done!


### DataFrame: Morning Notifications

In [33]:
morning_note = pd.read_csv('digital-marshmallow-morning_notification_time-v3_8.31.17.csv')
morning_note = remove_dupes(morning_note)

dupe_check(morning_note)

False

In [34]:
morning_note_attributes = [
    ['SAGE_morning_note_bl_-_', 'baseline']
]

morning_note = df_merge(morning_note, 
                        morning_note_attributes,
                        var_1='metadata.json.taskIdentifier')

In [35]:
final_check(morning_note, 'morning_note')

Done!


### DataFrame: PAM Multiple

In [36]:
pam_mult = pd.read_csv('digital-marshmallow-pam_multiple-v2_8.31.17.csv')
pam_mult = remove_dupes(pam_mult)

dupe_check(pam_mult)

False

In [37]:
pam_mult_attributes = [
    ['SAGE_pam_mult_bl_-_', 'baseline'],
    ['SAGE_pam_mult_21_-_', '21-day-assessment']
]

pam_mult = df_merge(pam_mult, 
                    pam_mult_attributes,
                    var_1='metadata.json.taskIdentifier')

In [38]:
final_check(pam_mult, 'pam_mult')

Done!


### Final Merge of All Sage Data

In [39]:
final_df = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                               on='externalId'), 
                  new_dfs.values())

In [40]:
dupe_check(final_df)

True

In [43]:
final_df.shape

(76, 1130)

In [41]:
os.chdir('/home/wraikes/Programming/Partnership/dmt/merged_data/')
final_df.to_csv('FINAL_SAGE.csv')